# Developing a model architecture for glioma classification

## Packages to use

In [2]:
## Imports go here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import nibabel as nib
import scikeras

from sklearn.model_selection import train_test_split

import tensorflow as tf

2023-03-10 10:26:42.193871: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 10:26:42.382774: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-10 10:26:42.382830: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-10 10:26:43.491392: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## Working with data

In [3]:
#load the dataframe
df = pd.read_pickle("../raw_data/Grade_ID_nii.pkl")

# the target columns have to be updated, it's only the fremework
df['Grade'] = df['Grade'].apply(lambda x: 0 if x == 'HGG' else 1)

# cut the image to smaller to match the location of brain
df['nii']= df['nii'].apply(lambda x: np.array(x[40:195,35:225,1:145]))

# split the Train and test
X_train, X_test, y_train, y_test = train_test_split(df['nii'], df['Grade'], test_size=0.2, random_state=42)

# change the X_train to match the input of tensorfolw
X_train = np.array([np.array(val) for val in X_train])


In [7]:
df['nii']= df['nii'].apply(lambda x: np.array(x[40:195,35:225,1:145]))


In [4]:
X_train = X_train.reshape(len(X_train), 155, 190, 144, 1)


## Model architecture - CNN

In [6]:
input_shape = (155, 190, 144, 1)
#around 295 observations

In [7]:
def initialize_model(kernel_size=(8,8,8)):
    model = Sequential()
    
    #Add convo layers to the model
    model.add(Conv3D(16, kernel_size=kernel_size, activation='relu', input_shape=input_shape))

    
    #Add a flatten layer
    model.add(Flatten())
    
    #Add layer with activation

    model.add(Dense(1, activation='sigmoid'))
    
    #Model compilation
    optim=Adam(learning_rate=0.01)
    model.compile(loss = 'binary_crossentropy',
                  optimizer = optim,
                  metrics = ['accuracy'])
    return model

In [13]:
#we need 1 kernel as we have 1 label for segmentation, kernel size = 3x3x3?

#initialize model
def initialize_model(dropout = 0.5, dense_1 = 50, \
    learning_rate = 0.01, kernel_size=(3,3,3), pool_size = (2,2,2)):
    model = Sequential()
    
    #Add convo layers to the model
    model.add(Conv3D(32, kernel_size=kernel_size, activation='relu', input_shape=input_shape))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Conv3D(64, kernel_size=kernel_size, activation='relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Conv3D(128, kernel_size=kernel_size, activation='relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    
    #Add a flatten layer
    model.add(Flatten())
    
    #maybe add age here for the second model
     
    #Add dense levels
    model.add(Dense(dense_1, activation='relu'))
    model.add(Dropout(dropout))

    
    #Add layer with activation
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    #Model compilation
    optim=Adam(learning_rate=learning_rate)
    model.compile(loss = 'binary_crossentropy',
                  optimizer = optim,
                  metrics = ['accuracy'])
    return model


In [8]:
#instantiate a model
model_seg = initialize_model()
model_seg.summary()

2023-03-10 10:27:28.396996: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-10 10:27:28.397047: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-10 10:27:28.397074: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (brain-project-cpu): /proc/driver/nvidia/version does not exist
2023-03-10 10:27:28.397411: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 148, 183, 137, 16  8208      
                             )                                   
                                                                 
 flatten (Flatten)           (None, 59368128)          0         
                                                                 
 dense (Dense)               (None, 1)                 59368129  
                                                                 
Total params: 59,376,337
Trainable params: 59,376,337
Non-trainable params: 0
_________________________________________________________________


In [ ]:
with tf.device('/device:gpu:1'):
    # Do something

In [11]:
#baseline model score

es = EarlyStopping(patience=3, restore_best_weights = True)
history = model_seg.fit(X_train, y_train,
                        epochs = 30,
                        batch_size = 32,
                        callbacks = [es],
                        validation_split = 0.2,
                        shuffle =True,
                        verbose = 1 
                        )

Epoch 1/30
2/8 [======>.......................] - ETA: 48:19 - loss: 1888.1198 - accuracy: 0.7656

In [35]:
#plot the learning curve
def plot_loss(history):
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(13,4))
    ax1.plot(history.history['loss'])
    ax1.plot(history.history['val_loss'])
    ax1.set_title('Model loss')
    ax1.set_ylabel('Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylim(ymin=0, ymax=200)
    ax1.legend(['Train', 'Validation'], loc='best')
    ax1.grid(axis="x",linewidth=0.5)
    ax1.grid(axis="y",linewidth=0.5)    
    
    ax2.plot(history.history['accuracy'])
    ax2.plot(history.history['val_accuracy'])
    ax2.set_title('Accuracy')
    ax2.set_ylabel('Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylim(ymin=0, ymax=20)
    ax2.legend(['Train', 'Validation'], loc='best')
    ax2.grid(axis="x",linewidth=0.5)
    ax2.grid(axis="y",linewidth=0.5)    

    plt.show()    

In [36]:
keras_estimator = KerasClassifier(build_fn = initialize_model, verbose = 1)

/var/folders/mz/tl4yflb54ks27kzvc57r1qvw0000gn/T/ipykernel_99831/1004603991.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_estimator = KerasClassifier(build_fn = initialize_model, verbose = 1)


In [37]:
estimator = Pipeline([('kc', keras_estimator)])

In [53]:
keras_estimator

{'verbose': 1,
 'build_fn': <function __main__.initialize_model(dropout=0.5, dense_1=50, learning_rate=0.01, kernel_size=(3, 3, 3), pool_size=(2, 2, 2))>}

In [54]:
estimator.get_params()#.get("kc__verbose")


{'memory': None,
 'steps': [('kc',
   <keras.wrappers.scikit_learn.KerasClassifier at 0x15feb1bd0>)],
 'verbose': False,
 'kc': <keras.wrappers.scikit_learn.KerasClassifier at 0x15feb1bd0>,
 'kc__verbose': 1,
 'kc__build_fn': <function __main__.initialize_model(dropout=0.5, dense_1=50, learning_rate=0.01, kernel_size=(3, 3, 3), pool_size=(2, 2, 2))>}

In [39]:
#hyperparameters tuning
# Define the hyperparameters
param_grid = {
    'kc__dense_1': [20, 30, 50, 100],
    'kc__kernel_size': [(2,2,2),(3,3,3), (5,5,5), (7,7,7)],
    'kc__pool_size': [(2,2,2),(3,3,3)],
    'kc__batch_size':[8, 16, 32],
    'kc__dropout': [0.5, 0.4, 0.3, 0.2, 0.1, 0],
    'kc__learning_rate': [0.001, 0.01, 0.1]
}


In [40]:
kfold_splits = 5
grid = GridSearchCV(estimator=estimator,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=kfold_splits,  #StratifiedKFold(n_splits=kfold_splits, shuffle=True)
                    param_grid=param_grid,)

In [ ]:
grid_result = grid.fit(X, y) #callbacks=[tbCallBack]

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
#prediction with the final model 
model_seg.evaluate(X_test, y_test)

## Model acrhitecture - UNET

In [ ]:
#maybe we don't need it for now, will check later

## Model architecture - 2D CNN

In [ ]:
#maybe we don't need it for now, will check later